<a href="https://colab.research.google.com/github/Akshithkumarreddy/Machine_learning_AL/blob/main/%23001_test_CNN_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
!tar -xvf cifar-100-python.tar.gz

--2025-05-19 02:38:18--  https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169001437 (161M) [application/x-gzip]
Saving to: ‘cifar-100-python.tar.gz’

cifar-100-python.ta 100%[===================>] 161.17M  64.2MB/s    in 2.5s    

2025-05-19 02:38:21 (64.2 MB/s) - ‘cifar-100-python.tar.gz’ saved [169001437/169001437]

cifar-100-python/
cifar-100-python/file.txt~
cifar-100-python/train
cifar-100-python/test
cifar-100-python/meta


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os
from tqdm import tqdm

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

train_data = unpickle('cifar-100-python/train')
test_data = unpickle('cifar-100-python/test')
meta = unpickle('cifar-100-python/meta')

print(train_data.keys())
print(train_data[b'data'].shape)
print(test_data.keys())
print(test_data[b'data'].shape)
print(meta.keys())

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])
(50000, 3072)
dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])
(10000, 3072)
dict_keys([b'fine_label_names', b'coarse_label_names'])


In [ ]:
labels = meta[b'fine_label_names']
print(labels)
# labels = meta[b'coarse_label_names']
# print(labels)


[b'apple', b'aquarium_fish', b'baby', b'bear', b'beaver', b'bed', b'bee', b'beetle', b'bicycle', b'bottle', b'bowl', b'boy', b'bridge', b'bus', b'butterfly', b'camel', b'can', b'castle', b'caterpillar', b'cattle', b'chair', b'chimpanzee', b'clock', b'cloud', b'cockroach', b'couch', b'crab', b'crocodile', b'cup', b'dinosaur', b'dolphin', b'elephant', b'flatfish', b'forest', b'fox', b'girl', b'hamster', b'house', b'kangaroo', b'keyboard', b'lamp', b'lawn_mower', b'leopard', b'lion', b'lizard', b'lobster', b'man', b'maple_tree', b'motorcycle', b'mountain', b'mouse', b'mushroom', b'oak_tree', b'orange', b'orchid', b'otter', b'palm_tree', b'pear', b'pickup_truck', b'pine_tree', b'plain', b'plate', b'poppy', b'porcupine', b'possum', b'rabbit', b'raccoon', b'ray', b'road', b'rocket', b'rose', b'sea', b'seal', b'shark', b'shrew', b'skunk', b'skyscraper', b'snail', b'snake', b'spider', b'squirrel', b'streetcar', b'sunflower', b'sweet_pepper', b'table', b'tank', b'telephone', b'television', b'ti

In [ ]:
dataset_folder = "/content/cifar_100_images/train"
os.makedirs(dataset_folder, exist_ok=True)
for c in tqdm(range(len(train_data[b'data']))):
    data = train_data[b'data'][c]
    label = train_data[b'fine_labels'][c]
    r, g, b = data[0:1024].reshape(32, 32), data[1024:2048].reshape(32, 32), data[2048:].reshape(32, 32)
    img = np.dstack((r, g, b))
    plt.imsave(f"{dataset_folder}/{labels[label]}_{c}.png", img)

dataset_folder = "/content/cifar_100_images/test"
os.makedirs(dataset_folder, exist_ok=True)
for c in tqdm(range(len(test_data[b'data']))):
    data = test_data[b'data'][c]
    label = test_data[b'fine_labels'][c]
    r, g, b = data[0:1024].reshape(32, 32), data[1024:2048].reshape(32, 32), data[2048:].reshape(32, 32)
    img = np.dstack((r, g, b))
    plt.imsave(f"{dataset_folder}/{labels[label]}_{c}.png", img)

100%|██████████| 10000/10000 [00:06<00:00, 1607.72it/s]


In [ ]:
# 10000x3072
# 3072 = (Red)1024(H=32 x W=32) + (Green)1024(H=32 x W=32) + (Blue)1024(H=32 x W=32)

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Custom Dataset
class CustomImageDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),  # Converts to [C, H, W] and normalizes to [0,1]
        ])

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)

# CNN Model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 4, 3, padding=1, stride=2)   # 3x32x32 -> 4x16x16
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1, stride=2)   # 4x16x16 -> 8x8x8
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1, stride=2)  # 8x8x8 -> 16x4x4
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1, stride=2) # 16x4x4 -> 32x2x2
        self.conv5 = nn.Conv2d(32, 64, 3, padding=1, stride=2) # 32x2x2 -> 64x1x1

        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 100)  # 100 classes for CIFAR-100

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

def save_model(model, optimizer, path):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, path)
    print(f"Model succesfully saved to {path}")

def load_model(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print(f"Model succesfully loaded from {path}")
    return model, optimizer

# Training function
def train_epoch(train_dataloader, model, optimizer, criterion):
    train_loss = 0.0
    model.train()
    model = model.to(device)
    for _, (x, y) in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc="Training"):
        x = x.to(device)
        y = y.to(device)
        output = model(x)
        loss = criterion(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_dataloader)
    return model, optimizer, train_loss

# Testing function
def test_epoch(test_dataloader, model, criterion):
    test_loss = 0.0
    model.eval()
    model = model.to(device)
    with torch.no_grad():
        for _, (x, y) in tqdm(enumerate(test_dataloader), total=len(test_dataloader), desc="Testing"):
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            loss = criterion(output, y)
            test_loss += loss.item()
    test_loss /= len(test_dataloader)
    return test_loss

# Inference function
def inference(test_data_loader, model):
    model.eval()
    model = model.to(device)
    predictions = []
    ground_truth = []
    with torch.no_grad():
        for _, (x, y) in tqdm(enumerate(test_data_loader), total=len(test_data_loader), desc="Inference"):
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            predictions.extend(output.argmax(dim=1).detach().cpu().numpy())
            ground_truth.extend(y.detach().cpu().numpy())
    accuracy = (np.array(predictions) == np.array(ground_truth)).sum() / len(ground_truth)
    return accuracy

# Training loop
def train_test(train_dataloader, test_dataloader, model, optimizer, criterion):

    previous_accuracy = 0.0

    for epoch in range(10):
        model = model.to(device)
        if os.path.exists("model_current.pth"):
            model, optimizer = load_model(model, optimizer, f"model_current.pth")

        model, optimizer, train_loss = train_epoch(train_dataloader, model, optimizer, criterion)
        print(f"Epoch {epoch + 1}: Training Loss = {train_loss:.4f}")

        save_model(model, optimizer, f"model_current.pth")
        model, optimizer = load_model(model, optimizer, f"model_current.pth")

        test_loss = test_epoch(test_dataloader, model, criterion)
        print(f"Epoch {epoch + 1}: Test Loss = {test_loss:.4f}")
        accuracy = inference(test_dataloader, model)
        print(f"Epoch {epoch + 1}: Accuracy = {accuracy * 100:.2f}%")
        if accuracy > previous_accuracy:
            previous_accuracy = accuracy
            save_model(model, optimizer, f"model_best.pth")
        print("-" * 50)

# Main function
def main():
    data_folder = "cifar_100_images"
    train_folder = os.path.join(data_folder, "train")
    test_folder = os.path.join(data_folder, "test")

    train_x = [os.path.join(train_folder, f) for f in os.listdir(train_folder)]
    test_x = [os.path.join(test_folder, f) for f in os.listdir(test_folder)]

    train_labels = [f.split('_')[0] for f in os.listdir(train_folder)]
    test_labels = [f.split('_')[0] for f in os.listdir(test_folder)]

    le = LabelEncoder()
    train_labels = le.fit_transform(train_labels)
    test_labels = le.transform(test_labels)

    train_data = [(train_x[i], train_labels[i]) for i in range(len(train_x))]
    test_data = [(test_x[i], test_labels[i]) for i in range(len(test_x))]

    train_dataset = CustomImageDataset(train_data)
    test_dataset = CustomImageDataset(test_data)

    train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    model = CNN()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    train_test(train_dataloader, test_dataloader, model, optimizer, criterion)

if __name__ == "__main__":
    main()


Model succesfully loaded from model_current.pth


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Training: 100%|██████████| 391/391 [00:17<00:00, 21.94it/s]

Epoch 1: Training Loss = 3.5503
Model succesfully saved to model_current.pth
Model succesfully loaded from model_current.pth



Testing: 100%|██████████| 79/79 [00:02<00:00, 27.35it/s]

Epoch 1: Test Loss = 3.5628


...

Epoch 7: Test Loss = 3.3804



Inference: 100%|██████████| 79/79 [00:02<00:00, 26.49it/s]


Epoch 7: Accuracy = 18.10%
Model succesfully saved to model_best.pth
--------------------------------------------------
Model succesfully loaded from model_current.pth


Training:  56%|█████▋    | 220/391 [00:09<00:07, 22.24it/s]


KeyboardInterrupt: 